# Binary-classification Task

In [1]:
import numpy
import pandas # or use `polars`

## Data Loading and Processing

### Load Dataset from XLS File by IO Process

In [2]:
df = pandas.read_excel('../data/credit_card_clients.xls', header=1, index_col=0)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Convert Any Format of Data to Numpy

In [3]:
df = df.to_numpy()

### Obtain Features and Labels for Subsequent Training

In [4]:
features, labels = df[:,:-1], df[:,-1]
features, labels

(array([[ 20000,      2,      2, ...,      0,      0,      0],
        [120000,      2,      2, ...,   1000,      0,   2000],
        [ 90000,      2,      2, ...,   1000,   1000,   5000],
        ...,
        [ 30000,      1,      2, ...,   4200,   2000,   3100],
        [ 80000,      1,      3, ...,   1926,  52964,   1804],
        [ 50000,      1,      2, ...,   1000,   1000,   1000]], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 1], dtype=int64))

In [5]:
features.shape, labels.shape

((30000, 23), (30000,))

## Machine Learning Process

### Load Perming and Config Hyperparameters

In [6]:
import perming
main = perming.Box(23, 2, (50,), batch_size=8, activation='relu', inplace_on=True, solver='adam', learning_rate_init=0.01)
# main = perming.Binarier(23, (50,), batch_size=8, activation='relu', solver='adam', learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Binary-classification'](23, (50,), batch_size=8, activation='relu', solver='adam', learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=23, out_features=50, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=50, out_features=2, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', CrossEntropyLoss()),
             ('batch_size', 8),
             ('solver',
              Adam (
              Parameter Group 0
                  amsgrad: False
                  betas: (0.9, 0.99)
                  eps: 1e-08
                  lr: 0.01
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### DataLoader from Numpy with Multi-threaded

In [7]:
main.data_loader(features, labels, random_seed=0)

### Training Stage and Accelerated Validation

In [8]:
main.train_val(num_epochs=1, interval=100)

Epoch [1/1], Step [100/3000], Training Loss: 693.3782, Validation Loss: 175.4494
Epoch [1/1], Step [200/3000], Training Loss: 3.4365, Validation Loss: 36.7887
Epoch [1/1], Step [300/3000], Training Loss: 4.2890, Validation Loss: 7.5508
Epoch [1/1], Step [400/3000], Training Loss: 0.4256, Validation Loss: 6.5157
Epoch [1/1], Step [500/3000], Training Loss: 0.2876, Validation Loss: 2.8152
Epoch [1/1], Step [600/3000], Training Loss: 0.5635, Validation Loss: 1.8385
Epoch [1/1], Step [700/3000], Training Loss: 0.5680, Validation Loss: 1.3541
Epoch [1/1], Step [800/3000], Training Loss: 0.2426, Validation Loss: 0.6416
Epoch [1/1], Step [900/3000], Training Loss: 0.5324, Validation Loss: 0.6116
Epoch [1/1], Step [1000/3000], Training Loss: 0.4133, Validation Loss: 0.5880
Epoch [1/1], Step [1100/3000], Training Loss: 0.5669, Validation Loss: 0.5473
Epoch [1/1], Step [1200/3000], Training Loss: 0.5659, Validation Loss: 0.5472
Epoch [1/1], Step [1300/3000], Training Loss: 0.7329, Validation Los

### Test Models with Accuracy and Correct Labels

In [9]:
main.test()

loss of Box on the 3000 test dataset: 1.4292031526565552. accuracy: 76.1667 %


OrderedDict([('problem', 'classification'),
             ('num_classes', 2),
             ('column', ('label name', ('true numbers', 'total numbers'))),
             ('labels', {0: [2285, 3000], 1: [2285, 3000]}),
             ('sorted', [(0, [2285, 3000]), (1, [2285, 3000])]),
             ('loss',
              {'train': 0.39861032366752625,
               'val': 1.115988850593567,
               'test': 1.4292031526565552})])

### Save Model Parameters to Models Folder

In [10]:
main.save(show=False, dir='../models/credit.ckpt')

### Load Model Parameters from Models Foler

In [11]:
main.load(show=False, dir='../models/credit.ckpt')

### Test with Loaded Parameters of Tune Model

In [12]:
main.test()
# main = perming.Box(23, 2, (50,), batch_size=8, activation='relu', inplace_on=True, solver='adam', learning_rate_init=0.01)
# main = perming.Binarier(23, 2, (50,), batch_size=8, activation='relu', solver='adam', learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Binary Classifier'](23, 2, (50,), batch_size=8, activation='relu', solver='adam', learning_rate_init=0.01)
# main.print_config()

loss of Box on the 3000 test dataset: 1.4292024374008179. accuracy: 76.1667 %


OrderedDict([('problem', 'classification'),
             ('num_classes', 2),
             ('column', ('label name', ('true numbers', 'total numbers'))),
             ('labels', {0: [2285, 3000], 1: [2285, 3000]}),
             ('sorted', [(0, [2285, 3000]), (1, [2285, 3000])]),
             ('loss',
              {'train': 0.39861032366752625,
               'val': 1.115988850593567,
               'test': 1.4292024374008179})])